In [1]:
import wandb
from dotenv import load_dotenv
import os
import numpy as np
import json
from copy import deepcopy
import tensorflow as tf
import keras_tuner as kt
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
from sklearn.model_selection import train_test_split
import sys
sys.path.append(os.path.abspath('../'))
import utils.dataset_loader as dataset_loader
import utils.my_model as model_builder
# Load the .env file
load_dotenv()

2025-05-08 18:31:22.595058: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 18:31:23.459552: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 18:31:23.879747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746721884.494540    3098 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746721884.635440    3098 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746721885.593829    3098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

True

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2025-05-08 18:31:40.175077: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
# Optionally fetch the key (for debugging or explicit control)
wandb_api_key = os.getenv("WANDB_API_KEY")

# Check if the key is available
if wandb_api_key is None:
    print("WANDB_API_KEY not found in environment variables.")
else:
    wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/miroslav/.netrc
wandb: Currently logged in as: xtodorovic (mt-thesis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# Paths 
DATASETS_PATH = '../datasets'
random_dataset = dataset_loader.load_dataset_files_with_cache(DATASETS_PATH, cache_path=f"{DATASETS_PATH}/cache/random_dataset_cache.pkl")

Checking for cache at: ../datasets/cache/random_dataset_cache.pkl
Loading datasets from cache: ../datasets/cache/random_dataset_cache.pkl


In [5]:
traces = dataset_loader.get_trace_matrix(random_dataset)  # shape (n_traces, n_samples)
print(f"Loaded traces: {traces.shape}")


Loaded traces: (10000, 5000)


In [ ]:
def train_model_fixed(traces, labels, model_out_path, augment=False, config=None):
    """
    Train a fixed model using given trace and label data.
    """
    if augment:
        traces = traces * np.random.uniform(0.9, 1.1)

    if len(traces.shape) == 2:
        traces = np.expand_dims(traces, axis=-1)

    x_train, x_val, y_train, y_val = train_test_split(
        traces, labels, test_size=0.2, shuffle=True, random_state=42
    )

    input_shape = x_train.shape[1:]
    num_classes = len(np.unique(y_train))
    
    print(f"Input shape: {input_shape}, Number of classes: {num_classes}")
    model = model_builder.build_cnn_model(input_shape[0], num_classes)

    run = wandb.init(
        entity="mt-thesis",
        project="model-training",
        name=os.path.splitext(os.path.basename(model_out_path))[0],
        config=config,
        reinit=True
    )
    
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-5,
        verbose=1
    )
    
    with tf.device('/gpu:0'):
        model.fit(
            x_train, y_train,
            epochs=1, 
            batch_size=64, 
            validation_data=(x_val, y_val),
            verbose=1,
            callbacks=[
                WandbMetricsLogger(),
                WandbModelCheckpoint(
                    filepath=model_out_path,
                    save_best_only=True,
                    monitor='val_accuracy',
                    mode='max'
                ),
                lr_scheduler
            ])

    model.save(model_out_path)
    run.finish()


In [7]:
def load_trace_ranges(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    global_range = tuple(data["global"])

    # Convert all per-nibble ranges to list-of-tuples
    per_nibble = {
        int(k): [tuple(r) for r in v] for k, v in data["per_nibble"].items()
    }

    return global_range, per_nibble

In [8]:
def get_traces_for_ranges(traces, ranges):
    if isinstance(ranges, tuple):  # single (start, end)
        return traces[:, ranges[0]:ranges[1]]
    elif isinstance(ranges, list):  # list of (start, end)
        return np.concatenate([traces[:, start:end] for start, end in ranges], axis=1)
    else:
        raise ValueError("Invalid trace range format.")

In [9]:
def get_wandb_config(algo, use_pois, poi_type, trace_mode, nibble, augment):
    wandb_config = {
        "algorithm": algo,
        "use_pois": use_pois,
        "poi_type": poi_type if use_pois else None,
        "trace_mode": trace_mode if not use_pois else None,
        "nibble": nibble,
        "augmentation": augment
    }
    return wandb_config

In [14]:
def run_all_trainings(base_dataset_dir, output_dir):
    algorithms = ["PRESENT", "AES"]
    use_pois_options = [True, False]
    poi_types = ["global", "per_nibble"]
    augmentation_options = [True, False]

    for algo in algorithms:
        label = "nibble" if algo == "PRESENT" else "byte" 
        dataset_path = os.path.join(base_dataset_dir, algo.lower())
        trace_ranges_path = os.path.join(dataset_path, "ranges.json")
        global_trace_range, per_nibble_trace_ranges = load_trace_ranges(trace_ranges_path)
        for use_pois in use_pois_options:
            for augment in augmentation_options:
                if use_pois:
                    for poi_type in poi_types:
                        for idx in range(1):
                            print(f"Training {algo} | POIs={poi_type} | {label}={idx} | Aug={augment}")
                            pois_path = f"{dataset_path}/pois/{'global' if poi_type == 'global' else f'{label}_{idx}_pois'}.npy"
                            pois = np.load(pois_path)
                            _traces = traces.copy()
                            _traces = _traces[:, pois]
                            print(f"Loaded POIs: {_traces.shape}")  # (n_traces, n_pois, 1)
                            labels = np.load(f"{dataset_path}/{label}_{idx}_labels.npy")

                            out_path = f"{output_dir}/{algo}/{poi_type}_POI_n{idx}_aug{int(augment)}.keras"
                            config = get_wandb_config(algo, use_pois, poi_type, None, idx, augment)
                            train_model_fixed(_traces, labels, out_path, augment, config)

                else:
                    for trace_mode, trace_range in [("global", global_trace_range), ("per_nibble", per_nibble_trace_ranges)]:
                        for idx in range(1):
                            print(f"Training {algo} | {trace_mode} trace | {label}={idx} | Aug={augment}")

                            if trace_mode == "per_nibble":
                                ranges = trace_range[idx]  # list of (start, end)
                            else:
                                ranges = trace_range          # single (start, end)

                            # Get the traces for the specified ranges
                            _traces = traces.copy()
                            trace_subset = get_traces_for_ranges(_traces, ranges)
                            print(f"Subset shape: {trace_subset.shape}")
                            labels = np.load(f"{dataset_path}/{label}_{idx}_labels.npy")

                            out_path = f"{output_dir}/{algo}/{trace_mode}_range_n{idx}_aug{int(augment)}.keras"
                            config = get_wandb_config(algo, use_pois, None, trace_mode, idx, augment)
                            train_model_fixed(trace_subset, labels, out_path, augment, config)


In [11]:
DATASET_DIR = "../dataset"

In [12]:
num_classes = 16

In [13]:
run_all_trainings(DATASET_DIR, "trained")

Training PRESENT | POIs=global | nibble=0 | Aug=True
Loaded POIs: (10000, 702)
Training PRESENT | POIs=per_nibble | nibble=0 | Aug=True
Loaded POIs: (10000, 47)
Training PRESENT | POIs=global | nibble=0 | Aug=False
Loaded POIs: (10000, 702)
Training PRESENT | POIs=per_nibble | nibble=0 | Aug=False
Loaded POIs: (10000, 47)
Training PRESENT | global trace | nibble=0 | Aug=True
Subset shape: (10000, 2050)
Training PRESENT | per_nibble trace | nibble=0 | Aug=True
Subset shape: (10000, 350)
Training PRESENT | global trace | nibble=0 | Aug=False
Subset shape: (10000, 2050)
Training PRESENT | per_nibble trace | nibble=0 | Aug=False
Subset shape: (10000, 350)
Training AES | POIs=global | byte=0 | Aug=True
Loaded POIs: (10000, 800)
Training AES | POIs=per_nibble | byte=0 | Aug=True
Loaded POIs: (10000, 55)
Training AES | POIs=global | byte=0 | Aug=False
Loaded POIs: (10000, 800)
Training AES | POIs=per_nibble | byte=0 | Aug=False
Loaded POIs: (10000, 55)
Training AES | global trace | byte=0 | A